In [1]:
import pandas as pd
import heapq
from functools import total_ordering

In [4]:
clients = pd.read_csv("DataSets\main-set\input_clients.csv")
display(clients)

,ClientID,Currencies,PositionCheck,Rating
0,CLIENT_100,"SGD,JPY",Y,1
1,CLIENT_101,"USD,JPY,SGD",N,10
2,CLIENT_102,SGD,Y,3
3,CLIENT_103,"SGD,USD,JPY",N,8
4,CLIENT_104,"SGD,JPY,USD",Y,9
5,CLIENT_105,"JPY,USD,SGD",Y,3
6,CLIENT_106,JPY,Y,4
7,CLIENT_107,JPY,Y,7
8,CLIENT_108,"SGD,USD,JPY",Y,5
9,CLIENT_109,"USD,SGD,JPY",Y,3


In [5]:
instruments = pd.read_csv("DataSets\main-set\input_instruments.csv")
display(instruments)

,InstrumentID,Currency,LotSize
0,INST_000,SGD,1000
1,INST_001,USD,100
2,INST_002,JPY,1
3,INST_003,SGD,1
4,INST_004,USD,100
5,INST_005,JPY,1000
6,INST_006,SGD,1
7,INST_007,USD,1
8,INST_008,JPY,1000
9,INST_009,SGD,1000


In [6]:
orders = pd.read_csv("DataSets\main-set\input_orders.csv")
display(orders)

,Time,OrderID,Instrument,Quantity,Client,Price,Side
0,09:00:00,ORDER_10,INST_000,17500.0,CLIENT_107,Market,Sell
1,09:00:00,ORDER_1430,INST_000,3000.0,CLIENT_111,Market,Sell
2,09:00:00,ORDER_1860,INST_000,46000.0,CLIENT_113,Market,Sell
3,09:00:00,ORDER_20,INST_001,4350.0,CLIENT_106,Market,Sell
4,09:00:00,ORDER_2430,INST_001,8700.0,CLIENT_103,Market,Buy
...,...,...,...,...,...,...,...
9995,16:09:00,ORDER_8859,INST_007,62.0,CLIENT_104,99.6,Sell
9996,16:09:00,ORDER_9429,INST_008,59000.0,CLIENT_112,100.1,Buy
9997,16:09:00,ORDER_9859,INST_008,94000.0,CLIENT_109,100.9,Buy
9998,16:09:00,ORDER_10429,INST_009,31000.0,CLIENT_105,100.1,Sell


In [7]:
class Client:
    def __init__(self, ID, Currencies, PosCheck, Rating, InstrumentsNameList):
        self.ID = ID
        self.Currencies = Currencies
        self.PosCheck = PosCheck
        self.Rating = Rating
        self.NetPosition = {}
        for i in InstrumentsNameList:
            self.NetPosition[i] = 0

In [21]:
@total_ordering
class Order:
    def __init__(self, Time, orderID, Instrument, Quantity, Client, Price, Side):
        self.Time =Time
        self.orderID = orderID
        self.Instrument = Instrument
        self.Quantity = Quantity
        self.Client = Client
        self.Price = Price
        self.Side = Side
        self.Rejection = False
        self.RejectionRsn = None
        self.ClientRating = None

    
    def __lt__(self, other):


        if (self.Price != other.Price):
            # Check for -1 for market order
            if (self.Price == -1): return True
            elif (other.Price == -1): return False
            if (self.Price < other.Price & self.Side == "Sell"):
                return True
            elif (self.Price > other.Price & self.Side == "Sell"):
                return False
            elif (self.Price > other.Price & self.Side == "Buy"):
                return True
            elif (self.Price < other.Price & self.Side == "Buy"):
                return False

        if (self.ClientRating > other.ClientRating):
            return True
        elif (self.ClientRating < other.ClientRating):
            return False
        elif (self.Time < other.Time):
            return True
        elif (self.Time >= other.Time):
            return False

    # def __gt__(self, other):
    #     if (self.Price > other.Price & self.Side == "Sell"):
    #         return True
    #     elif (self.Price < other.Price & self.Side == "Sell"):
    #         return False
    #     elif (self.Price < other.Price & self.Side == "Buy"):
    #         return True
    #     elif (self.Price > other.Price & self.Side == "Buy"):
    #         return False
    #     elif (self.ClientRating < other.ClientRating):
    #         return True
    #     elif (self.ClientRating > other.ClientRating):
    #         return False
    #     elif (self.Time > other.Time):
    #         return True
    #     elif (self.Time <= other.Time):
    #         return False

    def CheckOrder(self, Client, InstrumentNameList, InstrumentList):
        
        if (self.Instrument not in InstrumentNameList):
            #InstrumentNameList is just the InstrumentID column
            self.Rejection = True
            self.RejectionRsn = "REJECTED-INSTRUMENT NOT FOUND"
            return False
        else:
            for x in InstrumentList:
                if (x.ID == self.Instrument):
                    Instrument = x
        if (Instrument.Currency not in Client.Currencies):
            #REJECT MISMATCH CURRENCY
            self.Rejection = True
            self.RejectionRsn = "REJECTED-MISMATCH CURRENCY"
            return False
        elif (self.Quantity % Instrument.LotSize):
            self.Rejection = True
            self.RejectionRsn = "REJECTED-INVALID LOT SIZE"
            return False
        elif ((Client.PosCheck == "Y") & 
              (self.Side == "Sell") & 
              ((Client.NetPosition[self.Instrument] - self.Quantity) < 0)):
            self.Rejection = True
            self.RejectionRsn = "REJECTED-POSITION CHECK FAILED" + str(Client.NetPosition)
            return False

        self.ClientRating = Client.Rating

        return True

In [22]:
class Instrument:
    def __init__(self, ID, Currency, LotSize):
        self.ID = ID
        self.Currency = Currency
        self.LotSize = LotSize
        self.OpenPrice = None
        self.ClosePrice = None
        self.TotalVolume = 0
        self.PricexVol = 0
        self.DayHigh = 0
        self.DayLow = 0
        self.sellheap = []
        self.buyheap = []
    def heappush(self, NewOrder):
        if (Order.Side == "Sell"):
            heapq.heappush(self.sellheap, NewOrder)
        if (Order.Side == "Buy"):
            heapq.heappush(self.buyheap, NewOrder)
    def heapread(self, side):
        if (side == "Buy"):
            return self.buyheap[0]
        else:
            return self.sellheap[0]
    def heappop(self, side):
        if (side == "Buy"):
            return heapq.heappop(self.buyheap)
        else:
            return heapq.heappop(self.sellheap)
    def isEmpty(self, side):
        if (side == "Buy"):
            return len(self.buyheap) == 0
        else:
            return len(self.sellheap) == 0



In [23]:
rejection = pd.DataFrame(columns = ['Order Id', 'Rejection Reason'])

In [24]:
def createClientList(clients, instrumentNameList):
    clientList = []
    clientHash = {}
    for i in range(len(clients.index)):
        row = clients.iloc[i]
        client = Client(row['ClientID'], row['Currencies'], row['PositionCheck'], row['Rating'], instrumentNameList)
        clientList.append(client)
        clientHash[(row['ClientID'])] = client
    return clientList, clientHash

def findClient(clientId, clients):
    for client in clients:
        if client.ID == clientId:
            return client

In [25]:
def createInstrumentList(instruments):
    instrumentList = []
    instrumentNameList = []
    instrumentHash = {}
    for i in range(len(instruments.index)):
        row = instruments.iloc[i]
        instrument = Instrument(row['InstrumentID'], row['Currency'], row['LotSize'])
        instrumentList.append(instrument)
        instrumentNameList.append(row['InstrumentID'])
        instrumentHash[row["InstrumentID"]] = instrument
    return instrumentList, instrumentNameList, instrumentHash

In [26]:
instrumentList, instrumentNameList, tickerHash = createInstrumentList(instruments)
clientList, clientHash = createClientList(clients, instrumentNameList)

openOrderList = []
continuousOrderList = []
closeOrderList = []

In [27]:
def buildOrderBook(openOrderList, continuousOrderList, closeOrderList, clients, instruments, orders, rejection):
    for i in range(len(clients.index)):
        row = orders.iloc[i]
    
    orders['Time'] = pd.to_datetime(orders['Time'], format="%H:%M:%S")

    for i in range(1,len(orders.index)):
        row = orders.iloc[i]
        price = row['Price']
        if row['Price'] == 'Market':
            price = -1
        currClient = findClient(row['Client'], clientList)
        order = Order(row['Time'], row['OrderID'], row['Instrument'], row['Quantity'], row['Client'], price, row['Side'])
        if order.CheckOrder(currClient, instrumentNameList, instrumentList) == False:
                rejection.loc[len(rejection.index)] = (order.orderID, order.RejectionRsn)
        else:
            if row['Time'] < pd.to_datetime('9:30:00', format="%H:%M:%S"):
                if order.CheckOrder(currClient, instrumentNameList, instrumentList) == False:
                    rejection.loc[len(rejection.index)] = (order.orderID, order.RejectionRsn)
                else:
                    openOrderList.append(order)
            elif row['Time'] < pd.to_datetime('16:00:00', format="%H:%M:%S"):
                 continuousOrderList.append(order)
            else:
                 closeOrderList.append(order)
        # if row['Time'] < pd.to_datetime('9:30:00', format="%H:%M:%S"):
        #     if order.CheckOrder(currClient, instrumentNameList, instrumentList) == False:
        #         rejection.loc[len(rejection.index)] = (order.orderID, order.RejectionRsn)
        #     else:
        #         orderList.append(order)
        # else:
        #     if order.CheckOrder(currClient, instrumentNameList, instrumentList) == False:
        #         rejection.loc[len(rejection.index)] = (order.orderID, order.RejectionRsn)
            
rejection = pd.DataFrame(columns = ['OrderId', 'Rejection Reason'])
buildOrderBook(openOrderList, continuousOrderList, closeOrderList, clients, instruments, orders, rejection)


in()


In [28]:
##buildOrderBook()
'''
orderList = [['9:00:01','A1','SIA','1500','A','Market','Buy']]
tickerHash = {"SIA":Instrument("SIA", "SGD", "100")}

clientHash = {"A": Client('A',"USD,SGD",'Y','1'),
              "B": Client('B',"USD,SGD,JPY",'N','2')}
def continuousTrading():
    for orderRow in orderList:
        order = Order(orderRow[0], orderRow[1], orderRow[2], orderRow[3], orderRow[4], orderRow[5], orderRow[6])
        ticker = tickerHash[order.Instrument]
        ticker.heappush(order)
        hasBuy = not ticker.isEmpty("Buy")
        hasSell = not ticker.isEmpty("Sell")
        bestBuyOrder = None
        if (hasBuy):
            bestBuyOrder = ticker.heappop("Buy")
        bestSellOrder = None
        if (hasSell):
            bestSellOrder = ticker.heappop("Sell")
        
        
        if (hasBuy and hasSell):
            bestBuyOrderPrice = bestBuyOrder.price
            bestSellOrderPrice = bestSellOrder.price
            
            if (bestBuyOrderPrice >= bestSellOrderPrice):
                minVol = min(bestBuyOrder.volume, bestSellOrder.volume)
                bestBuyOrder.volume = bestBuyOrder.volume - minVol 
                bestSellOrder.volume = bestSellOrder.volume - minVol 
                maxPrice = max(bestBuyOrder.price, bestSellOrder.price)
                ticker.PricexVol = ticker.PricexVol + minVol * maxPrice
                ticker.TotalVolume = ticker.TotalVolume + minVol
                buyClient = bestBuyOrder.client
                sellClient = bestSellOrder.client

                buyClient.NetPosition = client.NetPosition - minVol * maxPrice
                sellClient.NetPosition = client.NetPosition - minVol * maxPrice
                clientHash[buyClient.Client] = buyClient
                clientHash[sellClient.Client] = sellClient

                if (bestBuyOrder.volume == 0):
                    ticker.heappush(bestSellOrder)
                else:
                    ticker.heappush(bestBuyOrder)
            else:
                
                ticker.heappush(bestBuyOrder)
                ticker.heappush(bestSellOrder)
        else:
            if (hasBuy):
                ticker.heappush(bestBuyOrder)
            else:
                ticker.heappush(bestSellOrder)
'''

'\norderList = [[\'9:00:01\',\'A1\',\'SIA\',\'1500\',\'A\',\'Market\',\'Buy\']]\ntickerHash = {"SIA":Instrument("SIA", "SGD", "100")}\n\nclientHash = {"A": Client(\'A\',"USD,SGD",\'Y\',\'1\'),\n              "B": Client(\'B\',"USD,SGD,JPY",\'N\',\'2\')}\ndef continuousTrading():\n    for orderRow in orderList:\n        order = Order(orderRow[0], orderRow[1], orderRow[2], orderRow[3], orderRow[4], orderRow[5], orderRow[6])\n        ticker = tickerHash[order.Instrument]\n        ticker.heappush(order)\n        hasBuy = not ticker.isEmpty("Buy")\n        hasSell = not ticker.isEmpty("Sell")\n        bestBuyOrder = None\n        if (hasBuy):\n            bestBuyOrder = ticker.heappop("Buy")\n        bestSellOrder = None\n        if (hasSell):\n            bestSellOrder = ticker.heappop("Sell")\n        \n        \n        if (hasBuy and hasSell):\n            bestBuyOrderPrice = bestBuyOrder.price\n            bestSellOrderPrice = bestSellOrder.price\n            \n            if (bestBuyO

In [29]:
#Function to Match at 09:30

def Auction(Instrument, OoC, clientHash):
    matching = True
    currentPrice = 0
    volume = 0
    matchif = False
    InstrumentName = Instrument.ID
    if (Instrument.heapread('Buy')[-1].Price == -1):
        if (Instrument.heapread('Sell')[-1].Price == -1):
            currentPrice = "NULL"
            matchif = False
    
    while (matching):
        
        buyOrder = Instrument.heappop('Buy')
        sellOrder = Instrument.heappop('Sell')
        sellquant = sellOrder.Quantity
        buyquant = buyOrder.Quantity
        if (buyOrder.Price == -1 & sellOrder.Price == -1):
            matchif = True
            currentPrice = -1
        
        elif (buyOrder.Price == -1 & sellOrder.Price != -1):
            
            matchif = True
            if (sellOrder.Price > currentPrice): currentPrice = sellOrder.Price
        elif (buyOrder.Price != -1 & sellOrder.Price == -1):
            
            matchif = True
            if (sellOrder.Price < currentPrice): currentPrice = sellOrder.Price
        elif (buyOrder.Price >= sellOrder.Price):
            matchif = True
            currentPrice = buyOrder.Price

        if (matchif == False):
            matching = False
            continue
        else:
            if (buyquant > sellquant):
                sellquant = 0
                buyquant -= sellquant
                volume += sellquant
                 
            else:
                sellquant -= buyquant
                buyquant = 0
                volume += buyquant
        if (buyquant):
            buyOrder.Quantity = buyquant
            Instrument.heappush(buyOrder)
        if (sellquant):
            sellOrder.Quantity = sellquant
            Instrument.heappush(sellOrder)
    if (volume != 0):
        buyClient = clientHash[Instrument.buyOrder.Client]
        sellClient = clientHash[Instrument.sellOrder.Client]
        buyClient.NetPosition[InstrumentName] = buyClient.NetPosition[InstrumentName] + volume
        sellClient.NetPosition[InstrumentName] = sellClient.NetPosition[InstrumentName] - volume
    if (OoC == "Open"):
        Instrument.OpenPrice = currentPrice
        if (currentPrice != "NULL"):
            Instrument.DayHigh = currentPrice
            Instrument.DayLow = currentPrice
    else:
        Instrument.ClosePrice = currentPrice
        if (currentPrice != "NULL"):
            if (Instrument.DayHigh < currentPrice):
                Instrument.DayHigh = currentPrice
            if (Instrument.DayLow > currentPrice):
                Instrument.DayLow = currentPrice
    
    if (currentPrice != "NULL"):
        Instrument.VolxPrice = volume * currentPrice
        Instrument.TotalVolume += volume
        

In [ ]:
def main(openingLst, midLst, closeLst):
    #opening
    for order in openingLst:
        ##order = Order(orderRow[0], orderRow[1], orderRow[2], orderRow[3], orderRow[4], orderRow[5], orderRow[6])
        ticker = tickerHash[order.Instrument]
        ticker.heappush(order)
    
    for order in openingLst:
        Auction(order.Instrument, "Open", clientHash )

    #continuousTrading
    for order in midLst:
        ##order = Order(orderRow[0], orderRow[1], orderRow[2], orderRow[3], orderRow[4], orderRow[5], orderRow[6])
        #clientID = order.Client
        currClient = findClient(order.Client, clientList)
        if order.CheckOrder(currClient, instrumentNameList, instrumentList) == False:
                rejection.loc[len(rejection.index)] = (order.orderID, order.RejectionRsn)
                continue
        ticker = tickerHash[order.Instrument]
        ticker.heappush(order)
        hasBuy = not ticker.isEmpty("Buy")
        hasSell = not ticker.isEmpty("Sell")
        bestBuyOrder = None
        if (hasBuy):
            bestBuyOrder = ticker.heappop("Buy")
        bestSellOrder = None
        if (hasSell):
            bestSellOrder = ticker.heappop("Sell")
        
        
        if (hasBuy and hasSell):
            bestBuyOrderPrice = bestBuyOrder.price
            bestSellOrderPrice = bestSellOrder.price
            if (bestBuyOrderPrice == -1):
                bestBuyOrderPrice = ticker.OpenPrice
            
            if (bestSellOrderPrice == -1):
                bestSellOrderPrice = ticker.OpenPrice
            
            
            if (bestBuyOrderPrice >= bestSellOrderPrice):
                minVol = min(bestBuyOrder.volume, bestSellOrder.volume)
                bestBuyOrder.volume = bestBuyOrder.volume - minVol 
                bestSellOrder.volume = bestSellOrder.volume - minVol 
                maxPrice = max(bestBuyOrder.price, bestSellOrder.price)
                ticker.PricexVol = ticker.PricexVol + minVol * maxPrice
                ticker.TotalVolume = ticker.TotalVolume + minVol
                buyClient = bestBuyOrder.client
                sellClient = bestSellOrder.client

                buyClient.NetPosition = buyClient.NetPosition - minVol * maxPrice
                sellClient.NetPosition = sellClient.NetPosition - minVol * maxPrice
                clientHash[buyClient.Client] = buyClient
                clientHash[sellClient.Client] = sellClient

                if (bestBuyOrder.volume == 0):
                    ticker.heappush(bestSellOrder)
                else:
                    ticker.heappush(bestBuyOrder)
            else:
                
                ticker.heappush(bestBuyOrder)
                ticker.heappush(bestSellOrder)
        else:
            if (hasBuy):
                ticker.heappush(bestBuyOrder)
            else:
                ticker.heappush(bestSellOrder)

    #closing
    for order in closeLst:
        #order = Order(orderRow[0], orderRow[1], orderRow[2], orderRow[3], orderRow[4], orderRow[5], orderRow[6])
        currClient = findClient(order.Client, clientList)
        if order.CheckOrder(currClient, instrumentNameList, instrumentList) == False:
                rejection.loc[len(rejection.index)] = (order.orderID, order.RejectionRsn)
                continue
        ticker = tickerHash[order.Instrument]
        ticker.heappush(order)

    for order in closeLst:
        Auction(order.Instrument, "Close", clientHash )

rejection.to_csv('output_exchange_report.csv')